In [109]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [110]:
df = pd.read_csv("Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [111]:
X = df.iloc[:, 3:-1].values
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [112]:
y = df.iloc[:, -1].values
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [114]:
df.isnull().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

# Encoding Categorical Data

In [115]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [116]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], 
                       remainder='passthrough')
X = np.array(ct.fit_transform(X))
X

array([[1.0, 0.0, 0.0, ..., 1, 1, 101348.88],
       [0.0, 0.0, 1.0, ..., 0, 1, 112542.58],
       [1.0, 0.0, 0.0, ..., 1, 0, 113931.57],
       ...,
       [1.0, 0.0, 0.0, ..., 0, 1, 42085.58],
       [0.0, 1.0, 0.0, ..., 1, 0, 92888.52],
       [1.0, 0.0, 0.0, ..., 1, 0, 38190.78]], dtype=object)

In [117]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state = 0)

In [118]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [119]:
# ann = tf.keras.models.Sequential()

In [120]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Sample model architecture (adjust as needed)
ann = Sequential()
ann.add(Dense(units=6, activation='relu', input_dim=X_train.shape[1]))  # Hidden layer
ann.add(Dense(units=6, activation='softmax'))  # Output layer for 6 classes

# Compile the model with categorical crossentropy
ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# One-hot encode the target labels
y_train = to_categorical(y_train, num_classes=6)

# Train the model
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100


C:\Users\Huawei\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1861 - loss: 2.1233
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7105 - loss: 1.0463
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 999us/step - accuracy: 0.8029 - loss: 0.6616
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 971us/step - accuracy: 0.8053 - loss: 0.5379
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8033 - loss: 0.4922
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8143 - loss: 0.4609
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8122 - loss: 0.4559
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8102 - loss: 0.4397
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8198 - loss: 0.4303
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8112 - loss: 0.4418
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8230 - loss: 0.4215
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/

In [123]:
# Get predictions (as probabilities for each class)
y_pred = ann.predict(X_test)

# Convert probabilities to predicted class labels
y_pred_class = np.argmax(y_pred, axis=1)

# If y_test is already in integer class labels, just use it as is
# If y_test is one-hot encoded, convert it using np.argmax
# y_test_class = np.argmax(y_test, axis=1)  # REMOVE this line if y_test is 1D

# Reshape for concatenation
y_pred_class = y_pred_class.reshape(len(y_pred_class), 1)
y_test = y_test.reshape(len(y_test), 1)

# Concatenate predicted and true labels for comparison
print(np.concatenate((y_pred_class, y_test), axis=1))



63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 947us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


In [126]:
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

# 1. Predict the class probabilities (assuming multi-class classification)
y_pred = ann.predict(X_test)

# 2. Convert predicted probabilities to class labels (argmax to get highest probability class)
y_pred_class = np.argmax(y_pred, axis=1)

# 3. Ensure y_test is in the correct format
# If y_test is already in integer labels (shape = (2000,), skip this step)
# If y_test is one-hot encoded, convert it to class labels
# y_test_class = np.argmax(y_test, axis=1)  # REMOVE if y_test is already in class labels
y_test_class = y_test  # Assuming y_test is already in the correct format

# 4. Create confusion matrix
cm = confusion_matrix(y_test_class, y_pred_class)

# 5. Calculate accuracy
accuracy = accuracy_score(y_test_class, y_pred_class)

# 6. Print the confusion matrix and accuracy
print("Confusion Matrix:")
print(cm)

print(f"Accuracy: {accuracy * 100:.2f}%")


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Confusion Matrix:
[[1516   79]
 [ 232  173]]
Accuracy: 84.45%
